# Extra practice with groupby and loc
## (And a couple extra things too!)

Suppose you have a friend (let's call her Anna) that has just graduated after four years of college. During that time, she has been carefully keeping track of every expense she made during college, and now she wants to know a little about how much she spent.

However, Jill realizes that doing this by hand might be a bit difficult, so she calls you (knowing that you know `pandas` and can help).

Your task is to implement functions that help Anna answer some questions she has about her spending habits during college. Don't use loops, and make sure to use as much `pandas` as you can, since it's much faster.

Here is the dataset that Anna has been carefully curating:

In [4]:
import pandas as pd
from cse163_utils import assert_equals

expense_data = pd.read_csv('data/college_expenses.csv')

expense_data.head()

,Year,Quarter,Type,Amount
0,Freshman,Autumn,Tuition,3011.0
1,Freshman,Autumn,Housing,3988.0
2,Freshman,Autumn,Fees,304.0
3,Freshman,Autumn,Textbook,74.0
4,Freshman,Autumn,Textbook,65.0


## Problem #1

Write a function called expenses_by_year that accepts a DataFrame as a parameter and returns a Series of the total expenses summed by the year in which they were made.

In [5]:
def expenses_by_year(data):
    return data.groupby('Year')['Amount'].sum()

In [6]:
# Run this cell to test your code!
result = expenses_by_year(expense_data)
assert_equals(22844.00, result['Freshman'])
assert_equals(22839.09, result['Sophomore'])

## Problem #2

Write a function called tuition_by_year that accepts a DataFrame and returns a Series of the total tuition paid by year.

In [7]:
def tuition_by_year(data):
    is_tuition = data['Type'] == 'Tuition'
    data = data[is_tuition]
    return data.groupby('Year')['Amount'].sum()

In [8]:
# testing cell
result = tuition_by_year(expense_data)
assert_equals(9033.00, result['Freshman'])
assert_equals(9303.99, result['Senior'])

## Problem #3

Write a function called highest_total_personal that accepts a DataFrame and returns the largest amount spent on personal expenses in a year.

In [9]:
def highest_total_personal(data):
    is_personal = data['Type'] == 'Personal'
    data = data[is_personal]
    return data.groupby('Year')['Amount'].sum().max()

In [10]:
# testing cell
assert_equals(429.0, highest_total_personal(expense_data))

## Grouping on multiple columns

You can group by multiple columns in `.groupby`. Do this by passing a `List` instead of a single value, for example:

In [11]:
sample_data = pd.DataFrame([
    {'a': 50, 'b': 30, 'c': 25},
    {'a': 50, 'b': 36, 'c': 20},
    {'a': 50, 'b': 30, 'c': 18}
])
# groups the data into groups with identical 'a' and 'b' values,
# only if both columns are the same, then sums the 'c' column
sample_groups = sample_data.groupby(['a', 'b'])['c'].sum()
sample_groups

a   b 
50  30    43
    36    20
Name: c, dtype: int64

The object returned by this type of `.groupby` is a `Series`, where the index of the `Series` is a `Tuple`. Therefore, you can access columns by passing in a `Tuple`.

In [12]:
sample_groups[(50, 36)] # returns the value of the second row

20

## Problem #4

Write a function called highest_total_textbook_expense that accepts a DataFrame and returns the highest amount spent on textbooks in any 1 quarter (hint: you will have to group by multiple columns)

In [13]:
def highest_total_textbook_expense(data):
    is_textbook = data['Type'] == 'Textbook'
    data = data[is_textbook]
    return data.groupby(['Year','Quarter'])['Amount'].sum().max()

In [14]:
# testing cell
assert_equals(294, highest_total_textbook_expense(expense_data))

## Problem #5

Write a function called highest_num_personal that accepts a DataFrame and returns the highest number of personal expenses that were made in a year.

In [15]:
def highest_num_personal(data):
    is_personal = data['Type'] == 'Personal'
    data = data[is_personal]
    return data.groupby('Year')['Amount'].count()

In [16]:
result = highest_num_personal(expense_data)
assert_equals(16,  )
assert_equals(11, result['Senior'])

## Problem #6

Write a function called most_expensive_textbook that accepts a DataFrame and returns the year in which Jane bought the most expensive textbook.

In [17]:
def most_expensive_textbook(data):
    is_textbook = data['Type'] == 'Textbook'
    data = data[is_textbook]
    return data.loc[data['Amount'].idxmax(), 'Year']

In [18]:
assert_equals('Freshman', most_expensive_textbook(expense_data))

## Problem #7

Write a function called highest_variable that accepts a DataFrame and returns a Tuple of the year and quarter in which Jane spent the most on variable costs (textbooks and personal costs combined).

In [19]:
def highest_variable(data):
    is_textbook = data['Type'] == 'Textbook'
    is_personal = data['Type'] == 'Personal'
    data = data[is_textbook | is_personal]
    combined_costs = data.groupby(['Year','Quarter'])['Amount'].sum()
    return combined_costs.idxmax()

In [20]:
assert_equals(('Junior', 'Winter'), highest_variable(expense_data))

## One more fun thing with `.groupby`

One thing that is possible with `.groupby` is that you can aggregate all columns, not just one column, of a `DataFrame`. Do this by leaving out the second column of the `.groupby` call, like this:

In [21]:
# here is some sample data that we can work with
sample_data = pd.DataFrame([
    {'a': 50, 'b': 30, 'c': 25},
    {'a': 50, 'b': 36, 'c': 20},
    {'a': 53, 'b': 18, 'c': 14},
    {'a': 53, 'b': 11, 'c': 61},
    {'a': 50, 'b': 30, 'c': 18}
])

In [22]:
sample_regrouped = sample_data.groupby('a').sum()
sample_regrouped

,b,c
a,,
50,96,63
53,29,75


This looks bizarre - everything seems out of place! However, it means that column `a` is now the index of the `DataFrame`, and columns `b` and `c` are just normal columns.

In other words, this new structure is a `DataFrame` with columns of the original columns, except for the column that was grouped by (we grouped by column `a`, and columns `b` and `c` are left). Additionally, the index of the new `DataFrame` is made up of the original values of `a`.

In [23]:
# Returns the value in column 'b' where the value of 'a' is 50.
sample_regrouped.loc[50, 'b']

96

If you want to keep the new `a` column to be a column in the resultant `DataFrame`, you will need to reset the index, using the `reset_index` function of a `DataFrame`.

In [24]:
# After this call, 'a' is now a column of a new grouped DataFrame
sample_regrouped.reset_index()

,a,b,c
0,50,96,63
1,53,29,75


Now, the `a` column is just another column, and the index of the `DataFrame` starts at 0 and increases by 1 for each row, just like normal.